# Etherscan API

### Import Dependencies

In [162]:
# Import packages 
# Import packages 
import requests
import time
import json
import pandas as pd
from tqdm import tqdm
from IPython.display import display
from dotenv import dotenv_values
config = dotenv_values(".env")
APIKEY = config["APIKEY"]

### Import Unique Addresses Associated with a Project Contract Address

In [3]:
# List of all transaction for a contract
# https://etherscan.io/token/0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb
allTransaction01 = requests.get("http://api.etherscan.io" +
                                "/api?"+ 
                                "module=account&"+
                                "action=tokennfttx&"+
                                "contractaddress=0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb&"+
                                "page=1&"+
                                "offset=10000&"+
                                "startblock=0&"+
                                "endblock=99999999&"+
                                "sort=asc&"+
                                "apikey="+APIKEY
                               ).json()
# Limited to 10000 calls
# Hard code start block to 12514651 to obtain rest of data 
allTransaction02 = requests.get("http://api.etherscan.io" +
                                "/api?"+ 
                                "module=account&"+
                                "action=tokennfttx&"+
                                "contractaddress=0xa3aee8bce55beea1951ef834b99f3ac60d1abeeb&"+
                                "page=1&"+
                                "offset=10000&"+
                                "startblock="+ str(int(allTransaction01['result'][-1]['blockNumber'])) +
                                "&"+ 
                                "endblock=99999999&"+
                                "sort=asc&"+
                                "apikey=" + APIKEY
                               ).json()

In [4]:
allTransactions = allTransaction01['result'] + allTransaction02['result']


In [6]:
allTransactions_df = pd.DataFrame(allTransactions)


In [18]:
unique_addresses = set(list(allTransactions_df['from'].unique()) + list(allTransactions_df['to'].unique()))
len(unique_addresses)

7274

### EDA 3: Normal, Internal, and ERC721 Transactions

#### Utils

In [21]:
# Now that we have a list of all transactions 
# we will start looking at "from" and "to" tokens for each transactions(normal and internal). 

# Normal transaction by address 
def getNormalTranTxlist(address,APIKEY):
    resp = requests.get("http://api.etherscan.io"+
                    "/api?"+
                    "module=account&"+
                    "action=txlist&"+
                    "address="+ address+ "&"+
                    "startblock=0&"+
                    "endblock=99999999&"+
                    "sort=asc&"+
                    "apikey="+APIKEY).json()
    return resp

# Internal Transaction by address 
def getInternalTranTxlist(address,APIKEY):
    resp = requests.get("http://api.etherscan.io"+
                    "/api?"+
                    "module=account&"+
                    "action=txlistinternal"+
                    "&address="+ address+ "&"+
                    "startblock=0&"+
                    "endblock=99999999&"+
                    "sort=asc&"+
                    "apikey="+APIKEY).json()
    return resp

# All transactions by address 
def getTranTokennfttx(address,APIKEY):
    resp = requests.get(
                    "http://api.etherscan.io" +
                    "/api?"+ 
                    "module=account&"+
                    "action=tokennfttx&"+
                    "&address="+ address+ "&"+
                    "page=1&"+
                    "offset=10000&"+
                    "startblock=0&"+
                    "endblock=99999999&"+
                    "sort=asc&"+
                    "apikey="+APIKEY).json()
    return resp

7274

#### API Calls

In [27]:
## Internal Transactions From addresses 
start = time.time()
APIKEY = "J1JMXVUVU2BREMAQE39Q9YA8GDZ3W6VGKG"
internal_dict              = []
unsuccessful_internal_dict = []
for i in tqdm(unique_addresses):
    nest_start = time.time()
    if i != "0x0000000000000000000000000000000000000000":
        resp = getInternalTranTxlist(i,APIKEY)
        if resp['status'] != '1':
            print("Address: ",i,"\nMessage:" ,resp['message'])
            unsuccessful_internal_dict = unsuccessful_internal_dict + [{"address": i,
                                                             "message": resp['message']
                                                            }]
        else:
            # Internal Transactions
            internal_dict = internal_dict+[{"Address": i, "data": resp}]
    nest_end = time.time()
    fill = 0.200 - (nest_end-nest_start)
    if fill > 0:
        time.sleep(fill)

end = time.time()
print("Total Time Spent:", end - start)

  0%|                                          | 6/7274 [00:01<24:39,  4.91it/s]

Address:  0x5033205df272cffe22b930d7ed9b8b131d94980b 
Message: No transactions found


  0%|                                        | 8/7274 [00:02<1:00:00,  2.02it/s]

Address:  0x178571753ed490f4219b8e426b5316959ba9e002 
Message: No transactions found


  0%|                                         | 13/7274 [00:03<30:27,  3.97it/s]

...

Address:  0x0a21d2e057cfe9733101bdd9f6909fed5f779519 
Message: No transactions found


100%|██████████████████████████████████████▉| 7270/7274 [29:05<00:00,  4.82it/s]

Address:  0x9e55fc8a7c2439b9bef17b39ec42539210105d1f 
Message: No transactions found


100%|███████████████████████████████████████| 7274/7274 [29:06<00:00,  4.16it/s]

Total Time Spent: 1746.8706948757172


In [28]:
## Normal Transactions From addresses 
start = time.time()
APIKEY = "J1JMXVUVU2BREMAQE39Q9YA8GDZ3W6VGKG"
norm_dict              = []
unsuccessful_norm_dict = []
for i in tqdm(unique_addresses):
    nest_start = time.time()
    if i != "0x0000000000000000000000000000000000000000":
        # Normal Transactions
        resp = getNormalTranTxlist(i,APIKEY)        
        if resp['status'] != '1':
            print("Address: ",i,"\nMessage:" ,resp['message'])
            unsuccessful_norm_dict= unsuccessful_norm_dict + [{"address": i,
                                                             "message": resp['message']
                                                            }]
        else:
            norm_dict = norm_dict+[{"Address": i, "data": resp}]
            
    nest_end = time.time()
    fill = 0.200 - (nest_end-nest_start)
    if fill > 0:
        time.sleep(fill)

end = time.time()
print("Total Time Spent:", end - start)

  1%|▍                                        | 85/7274 [00:35<39:26,  3.04it/s]

Address:  0xa4f1bf827f3301193e8f1360c21e28941a7fedc2 
Message: No transactions found


  1%|▌                                       | 101/7274 [00:39<28:27,  4.20it/s]

Address:  0xb131b92edc6870ce3638200a351341cda028651b 
Message: No transactions found


  2%|▋                                       | 121/7274 [00:45<27:56,  4.27it/s]

...

Address:  0x2af2079c1bbbff82145fd61e6d3e4d47293d1495 
Message: No transactions found


100%|██████████████████████████████████████▉| 7267/7274 [42:34<00:01,  4.48it/s]

Address:  0xac27401f5ae4109e0fac53d90212de8218bef0b6 
Message: No transactions found


100%|███████████████████████████████████████| 7274/7274 [42:37<00:00,  2.84it/s]

Total Time Spent: 2557.7824177742004


In [29]:
## NFT Transactions From addresses 
start = time.time()
APIKEY = "J1JMXVUVU2BREMAQE39Q9YA8GDZ3W6VGKG"
nfttx_dict              = []
unsuccessful_nfttx_dict = []
for i in tqdm(unique_addresses):
    nest_start = time.time()
    if i != "0x0000000000000000000000000000000000000000":
        
        resp = getTranTokennfttx(i,APIKEY)
        if resp['status'] != '1':
            print("Address: ",i,"\nMessage:" ,resp['message'])
            unsuccessful_nfttx_dict= unsuccessful_nfttx_dict + [{"address": i,
                                                             "message": resp['message']
                                                  }]
        # NFT Transactions
        else:
            nfttx_dict = nfttx_dict+[{"Address": i, "data": resp}]
    
    nest_end = time.time()
    fill = 0.200 - (nest_end-nest_start)
    if fill > 0:
        time.sleep(fill)

end = time.time()
print("Total Time Spent:", end - start)

  1%|▌                                       | 101/7274 [00:26<30:21,  3.94it/s]

Address:  0xb131b92edc6870ce3638200a351341cda028651b 
Message: NOTOK


  2%|▋                                       | 118/7274 [00:31<31:04,  3.84it/s]

Address:  0x239c74655d4a1e9e4cb7aae25c5771878cefc77f 
Message: NOTOK


  2%|▊                                       | 142/7274 [00:37<30:41,  3.87it/s]

...

Address:  0xb97e26108290348f86f327bf1c52e8cace423fa6 
Message: NOTOK


 99%|██████████████████████████████████████▌| 7182/7274 [29:50<00:19,  4.80it/s]

Address:  0x760ff645a093bc97da57b6abea5ee7c8fa4b38b6 
Message: NOTOK


100%|███████████████████████████████████████| 7274/7274 [30:11<00:00,  4.02it/s]

Total Time Spent: 1811.0999789237976


##### Save to json

In [30]:
with open('nfttx_dict.json', 'w') as f:
    json.dump(nfttx_dict, f)
f.close()

with open('unsuccessful_nfttx_dict.json', 'w') as f:
    json.dump(unsuccessful_nfttx_dict, f)
f.close()

with open('internal_dict.json', 'w') as f:
    json.dump(internal_dict, f)
f.close()

with open('unsuccessful_internal_dict.json', 'w') as f:
    json.dump(unsuccessful_internal_dict, f)
f.close()

with open('norm_dict.json', 'w') as f:
    json.dump(norm_dict, f)
f.close()

with open('unsuccessful_norm_dict.json', 'w') as f:
    json.dump(unsuccessful_norm_dict, f)
f.close()
